In [2]:
import os
from model import Inventory_Policy
import warnings
import numpy as np 
import pandas as pd
warnings.filterwarnings('ignore')
from model import Inventory_Policy

#304213577 EGYPT
#304213061 Arab
sp = Inventory_Policy(304213061)
#initial model
sp.createModel()
sp.solve()
x1 = [sp.model.X[i]() for i in sp.model.i]
f1 = sp.model.ObjF()
print('X= ', x1,'\nOBJ= ',f1)
print(50*"*")

m_df = sp.opt_df.copy()
m_df = m_df[['name','stock','base_stock','order_quantity',]]
m_df['order_quantity'] += m_df['base_stock']
# Indirect Cost for cling SKUs
monthly_cling_proc = 196046.6756
dic = {}
for i in sp.df[sp.df.company != 'imported']['company'].unique():
    dic[i] = sp.df[sp.df.company != 'imported']\
    [sp.df[sp.df.company != 'imported']['company'] == i]\
    .annual_demand.sum() / sp.df[sp.df.company != 'imported'].annual_demand.sum()
    dic[i] *= monthly_cling_proc
# If model can order more....
if (x1 + sp.opt_df['stock']).sum() < (sp.opt_df['base_stock'] + sp.opt_df['order_quantity']).sum():
    #X1,COST,Profit
    if f1 < 1:
        sp.releaseStockConstraint
        sp.solve()
        
    m_df['X_1#CAR'] = [sp.model.X[i]() for i in sp.model.i]
    m_df['APIs'] = [np.nan for i in x1]
    m_df.loc[m_df.index[0],'APIs'] = 'TOTAL INCOME'
    m_df.loc[m_df.index[1],'APIs'] = 'TOTAL DIRECT COST'
    m_df.loc[m_df.index[2],'APIs'] = 'TOTAL INDIRECT COST'
    m_df.loc[m_df.index[3],'APIs'] = 'TOTAL PROFIT'
    m_df['Value(LE)'] = [np.nan for i in x1]
    m_df.loc[m_df.index[0],'Value(LE)'] =\
    sum(sp.opt_df.loc[:,'avg_price'] * x1) + sum(sp.opt_df.loc[:,'avg_price'] * sp.opt_df.loc[:,'stock'])
    m_df.loc[m_df.index[1],'Value(LE)'] =\
    sum(sp.opt_df.loc[:,'avg_cost'] * x1) + sum(sp.opt_df.loc[:,'avg_cost'] * sp.opt_df.loc[:,'stock'])
    m_df.loc[m_df.index[2],'Value(LE)'] =\
    dic[sp.opt_df.company.unique()[0]]
    m_df.loc[m_df.index[3],'Value(LE)'] =\
    m_df.loc[m_df.index[0],'Value(LE)'] - m_df.loc[m_df.index[1],'Value(LE)'] - m_df.loc[m_df.index[2],'Value(LE)']
    
    
    
    sp.SetNumCars(2)
    sp.createModel()
    sp.solve()
    if sp.model.ObjF() < 1:
        sp.releaseStockConstraint
        sp.solve()
    x2 = [sp.model.X[i]() for i in sp.model.i]
    f2 = sp.model.ObjF()
    print('X= ', x2,'\nOBJ= ',f2)

    print(50*"*")
    m_df['X_2#CAR'] = x2
    m_df['APIs '] = [np.nan for i in x2]
    m_df.loc[m_df.index[0],'APIs '] = 'TOTAL INCOME'
    m_df.loc[m_df.index[1],'APIs '] = 'TOTAL DIRECT COST'
    m_df.loc[m_df.index[2],'APIs '] = 'TOTAL INDIRECT COST'
    m_df.loc[m_df.index[3],'APIs '] = 'TOTAL PROFIT'
    m_df['Value (LE)'] = [np.nan for i in x2]
    m_df.loc[m_df.index[0],'Value (LE)'] =\
    sum(sp.opt_df.loc[:,'avg_price'] * x2) + sum(sp.opt_df.loc[:,'avg_price'] * sp.opt_df.loc[:,'stock'])
    m_df.loc[m_df.index[1],'Value (LE)'] =\
    sum(sp.opt_df.loc[:,'avg_cost'] * x2) + sum(sp.opt_df.loc[:,'avg_cost'] * sp.opt_df.loc[:,'stock'])
    m_df.loc[m_df.index[2],'Value (LE)'] =\
    dic[sp.opt_df.company.unique()[0]]
    m_df.loc[m_df.index[3],'Value (LE)'] =\
    m_df.loc[m_df.index[0],'Value (LE)'] - m_df.loc[m_df.index[1],'Value (LE)'] - m_df.loc[m_df.index[2],'Value (LE)']

    
    sp.releaseInventoryBalanceConstraint
    sp.releaseWeightConstraint
    sp.solve()
    x3 = [sp.model.X[i]() for i in sp.model.i]
    f3 = sp.model.ObjF()
    print('X= ', x3,'\nOBJ= ',f3)   
    ncars = int(np.ceil((sp.opt_df.loc[:,'weight'].values*[sp.model.X[i]() for i in sp.model.i]).sum() / sp.car_weight))
    print('#CARS',ncars)
    print(50*"*")
    if ncars > 2:
   
        m_df[f'X_{ncars}#CAR'] = x3
        m_df['APIs  '] = [np.nan for i in x3]
        m_df.loc[m_df.index[0],'APIs  '] = 'TOTAL INCOME'
        m_df.loc[m_df.index[1],'APIs  '] = 'TOTAL DIRECT COST'
        m_df.loc[m_df.index[2],'APIs  '] = 'TOTAL INDIRECT COST'
        m_df.loc[m_df.index[3],'APIs  '] = 'TOTAL PROFIT'
        m_df['Value (LE) '] = [np.nan for i in x3]
        m_df.loc[m_df.index[0],'Value (LE) '] =\
        sum(sp.opt_df.loc[:,'avg_price'] * x3) + sum(sp.opt_df.loc[:,'avg_price'] * sp.opt_df.loc[:,'stock'])
        m_df.loc[m_df.index[1],'Value (LE) '] =\
        sum(sp.opt_df.loc[:,'avg_cost'] * x3) + sum(sp.opt_df.loc[:,'avg_cost'] * sp.opt_df.loc[:,'stock'])
        m_df.loc[m_df.index[2],'Value (LE) '] =\
        dic[sp.opt_df.company.unique()[0]]
        m_df.loc[m_df.index[3],'Value (LE) '] =\
        m_df.loc[m_df.index[0],'Value (LE) '] - m_df.loc[m_df.index[1],'Value (LE) '] - m_df.loc[m_df.index[2],'Value (LE) ']

        #Rename Columns
        m_df.columns = ['name', 'stock', 'Min_INV', 'Max_INV', 'X_1#CAR', 'APIs',
           'Value(LE)', 'X_2#CAR', 'APIs ', 'Value (LE)', f'X_{ncars}CAR', 'APIs  ',
           'Value (LE) '] 
    else:
        #Rename Columns
        m_df.columns = ['name', 'stock', 'Min_INV', 'Max_INV', 'X_1#CAR', 'APIs',
           'Value(LE)', 'X_2#CAR', 'APIs ', 'Value (LE)'] 
else:
    #X1,COST,Profit
    m_df['X_1#CAR'] = x1
    m_df['APIs'] = [np.nan for i in x1]
    m_df.loc[m_df.index[0],'APIs'] = 'TOTAL INCOME'
    m_df.loc[m_df.index[1],'APIs'] = 'TOTAL DIRECT COST'
    m_df.loc[m_df.index[2],'APIs'] = 'TOTAL INDIRECT COST'
    m_df.loc[m_df.index[3],'APIs'] = 'TOTAL PROFIT'
    m_df['Value(LE)'] = [np.nan for i in x1]
    m_df.loc[m_df.index[0],'Value(LE)'] =\
    sum(sp.opt_df.loc[:,'avg_price'] * x1) + sum(sp.opt_df.loc[:,'avg_price'] * sp.opt_df.loc[:,'stock'])
    m_df.loc[m_df.index[1],'Value(LE)'] =\
    sum(sp.opt_df.loc[:,'avg_cost'] * x1) + sum(sp.opt_df.loc[:,'avg_cost'] * sp.opt_df.loc[:,'stock'])
    m_df.loc[m_df.index[2],'Value(LE)'] =\
    dic[sp.opt_df.company.unique()[0]]
    m_df.loc[m_df.index[3],'Value(LE)'] =\
    m_df.loc[m_df.index[0],'Value(LE)'] - m_df.loc[m_df.index[1],'Value(LE)'] - m_df.loc[m_df.index[2],'Value(LE)']
    #Rename Columns
    m_df.columns = ['name', 'stock', 'Min_INV', 'Max_INV', 'X_1#CAR', 'APIs','Value(LE)'] 

#make directory if not exist
file_name = "OUTPUT"
if not os.path.exists(file_name):
   os.makedirs(file_name)

with pd.ExcelWriter(F"OUTPUT\{sp.opt_df.company.unique()[0].upper()}.xlsx") as writer:
    m_df.to_excel(writer, sheet_name="Order")
    sp.opt_df.to_excel(writer, sheet_name="Data")

Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_MIPGap                   0
CPXPARAM_MIP_Tolerances_Integrality              1.0000000000000001e-09
Found incumbent of value 41963.889700 after 0.00 sec. (0.01 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 101 rows and 42 columns.
MIP Presolve modified 1 coefficients.
Aggregator did 1 substitutions.
Reduced MIP has 2 rows, 5 columns, and 10 nonzeros.
Reduced MIP has 1 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.08 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 2 rows, 5 columns, and 10 nonzeros.
Reduced MIP has 1 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.00 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root re